In [ ]:
!pip install -q moviepy faster-whisper
import os
import time
import shutil
import numpy as np
from tqdm import tqdm
from moviepy.editor import VideoFileClip
from faster_whisper import WhisperModel



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.7 MB/s eta 0:00:00


  if event.key is 'enter':



In [ ]:
def split_video_get_audio_segments(video_path, working_dir, segment_length=30, num_frames_per_segment=5, audio_output_format='mp3'):
    video_name = os.path.basename(video_path).split('.')[0]
    unique_timestamp = str(int(time.time() * 1000))
    segment_cache_path = os.path.join(working_dir, '_cache', video_name)

    if os.path.exists(segment_cache_path):
        shutil.rmtree(segment_cache_path)
    os.makedirs(segment_cache_path, exist_ok=True)

    segment_index2name, segment_times_info = {}, {}
    segment_index = 0

    with VideoFileClip(video_path) as video:
        total_duration = int(video.duration)
        start_times = list(range(0, total_duration, segment_length))

        if len(start_times) > 1 and (total_duration - start_times[-1]) < 5:
            start_times = start_times[:-1]

        for start in tqdm(start_times, desc="Splitting video (metadata + audio only)"):
            end = min(start + segment_length, total_duration)
            subvideo = video.subclip(start, end)

            frame_times = np.linspace(0, subvideo.duration, num_frames_per_segment, endpoint=False) + start

            segment_name = f"{unique_timestamp}-{segment_index}-{start}-{end}"
            segment_index2name[f"{segment_index}"] = segment_name
            segment_times_info[f"{segment_index}"] = {
                "frame_times": frame_times,
                "timestamp": (start, end)
            }

            audio_file = f"{segment_name}.{audio_output_format}"
            subvideo.audio.write_audiofile(
                os.path.join(segment_cache_path, audio_file), codec='mp3', verbose=False, logger=None
            )

            segment_index += 1

    return segment_index2name, segment_times_info

def speech_to_text_whisper(video_name, working_dir, segment_index2name, audio_output_format='mp3', whisper_model_size="medium"):
    model = WhisperModel(whisper_model_size, compute_type="auto")
    transcripts = {}
    audio_cache_path = os.path.join(working_dir, "_cache", video_name)

    for index in tqdm(segment_index2name, desc="Transcribing audio segments"):
        segment_name = segment_index2name[index]
        audio_file = os.path.join(audio_cache_path, f"{segment_name}.{audio_output_format}")

        segments, _ = model.transcribe(audio_file)
        transcript_text = ""
        for seg in segments:
            transcript_text += f"[{seg.start:.2f}s -> {seg.end:.2f}s] {seg.text}\n"
        transcripts[index] = transcript_text

    return transcripts



In [ ]:
video_path = "/content/CS50_Week1_S1_Datatypes.mp4"
working_dir = "/content"

segment_index2name, segment_times_info = split_video_get_audio_segments(
    video_path=video_path,
    working_dir=working_dir,
    segment_length=30,  # you can change this
    num_frames_per_segment=5,
    audio_output_format='mp3'
)

# Transcribe
transcripts = speech_to_text_whisper(
    video_name=os.path.basename(video_path).split('.')[0],
    working_dir=working_dir,
    segment_index2name=segment_index2name,
    audio_output_format='mp3',
    whisper_model_size="medium"

for idx, text in transcripts.items():
    print(f"\n--- Segment {idx} ---\n{text}")


Splitting video (metadata + audio only): 100%|██████████| 29/29 [00:43<00:00,  1.51s/it]
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Transcribing audio segments: 100%|██████████| 29/29 [55:21<00:00, 114.52s/it]


--- Segment 0 ---
[0.00s -> 8.64s]  All right, so let's talk about another thing that's kind of unique to C,
[8.64s -> 10.12s]  which is data types and variables.
[10.12s -> 12.44s]  And when I say unique to C, I really only mean in the context of,
[12.44s -> 14.60s]  if you've been a programmer for a really long time,
[14.60s -> 16.40s]  you've probably not worked with data types
[16.40s -> 18.60s]  if you've used modern programming languages.
[18.60s -> 20.88s]  Modern languages like PHP and JavaScript,
[20.88s -> 23.08s]  which we'll also see a little later on in the course,
[23.08s -> 25.64s]  you don't actually have to specify the data type of a variable
[25.64s -> 26.36s]  when you use it.
[26.36s -> 28.36s]  You just declare it and start using it.
[28.36s -> 31.16s]  If it's an integer, it knows it's an integer.


--- Segment 1 ---
[0.00s -> 2.00s]  If it's a character, it knows it's a character.
[2.00s -> 5.32s]  If it's a word, it knows it's a string, so-called.
[5.32s -> 7.2

In [ ]:
def split_video_for_audio_only(video_path, working_dir, segment_length=30, num_frames_per_segment=10, audio_output_format='mp3'):
    unique_timestamp = str(int(time.time() * 1000))
    video_name = os.path.basename(video_path).split('.')[0]
    video_segment_cache_path = os.path.join(working_dir, '_cache', video_name)
    if os.path.exists(video_segment_cache_path):
        shutil.rmtree(video_segment_cache_path)
    os.makedirs(video_segment_cache_path, exist_ok=True)

    segment_index2name, segment_times_info = {}, {}
    segment_index = 0

    with VideoFileClip(video_path) as video:
        total_video_length = int(video.duration)
        start_times = list(range(0, total_video_length, segment_length))

        if len(start_times) > 1 and (total_video_length - start_times[-1]) < 5:
            start_times = start_times[:-1]

        for start in tqdm(start_times, desc=f"Splitting Video {video_name}"):
            end = min(start + segment_length, total_video_length)
            subvideo = video.subclip(start, end)
            subvideo_length = subvideo.duration
            frame_times = np.linspace(start, end, num_frames_per_segment, endpoint=False)

            segment_key = f"{segment_index}"
            segment_name = f"{unique_timestamp}-{segment_index}-{start}-{end}"
            segment_index2name[segment_key] = segment_name
            segment_times_info[segment_key] = {"frame_times": frame_times, "timestamp": (start, end)}

            audio_path = os.path.join(video_segment_cache_path, f"{segment_name}.{audio_output_format}")
            subvideo.audio.write_audiofile(audio_path, codec='mp3', verbose=False, logger=None)

            segment_index += 1

    return video_name, segment_index2name, segment_times_info, video_segment_cache_path

def transcribe_audio_segments(video_name, cache_path, segment_index2name, segment_times_info, audio_output_format='mp3'):
    model = WhisperModel("large-v2", compute_type="auto")
    transcript_segments = []

    for index in tqdm(segment_index2name, desc=f"Transcribing Audio {video_name}"):
        segment_name = segment_index2name[index]
        audio_file = os.path.join(cache_path, f"{segment_name}.{audio_output_format}")
        start_offset = segment_times_info[index]["timestamp"][0]

        segments, _ = model.transcribe(audio_file)
        for seg in segments:
            full_start = seg.start + start_offset
            full_end = seg.end + start_offset
            transcript_segments.append({
                "video_segment": index,
                "video_timestamp_start": round(full_start, 2),
                "video_timestamp_end": round(full_end, 2),
                "text": seg.text.strip()
            })

    return transcript_segments

video_path = "/content/Machine Learning in 3 Minutes.mp4"
working_dir = "/content/working_dir"

video_name, segment_index2name, segment_times_info, cache_path = split_video_for_audio_only(
    video_path=video_path,
    working_dir=working_dir,
    segment_length=30,
    num_frames_per_segment=10
)

transcripts = transcribe_audio_segments(
    video_name=video_name,
    cache_path=cache_path,
    segment_index2name=segment_index2name,
    segment_times_info=segment_times_info,
    audio_output_format='mp3'
)

for seg in transcripts:
    print(f"[{seg['video_timestamp_start']}s -> {seg['video_timestamp_end']}s] {seg['text']}")


Splitting Video Machine Learning in 3 Minutes: 100%|██████████| 7/7 [00:04<00:00,  1.48it/s]


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Transcribing Audio Machine Learning in 3 Minutes: 100%|██████████| 7/7 [00:20<00:00,  2.88s/it]

[0.0s -> 4.56s] Hi, welcome to E2E clouds machine learning series.
[4.56s -> 8.2s] In this video, we learn machine learning in 3 minutes.
[8.2s -> 11.76s] Any AI algorithm is really just a massive math problem.
[11.76s -> 16.12s] You take a bunch of inputs, perform hundreds of equations on those inputs, and arrive at
[16.12s -> 18.32s] the desired output.
[18.32s -> 22.14s] Each one of these circles represents a math equation.
[22.14s -> 27.16s] Each line between two circles represents a variable or weight that is learned over the
[27.16s -> 29.76s] course of a process called training.
[29.76s -> 30.3s] Let's learn.
[30.0s -> 35.0s] for different strengths of relations between these circles, or neurons, to be determined
[35.0s -> 40.08s] that can help a neural network figure out a final classification or prediction.
[40.08s -> 43.84s] The stronger the connection, the higher the weight value.
[43.84s -> 48.68s] In order to learn the values of the weights that give a machine learning alg

In [ ]:
def extract_audio(video_path, audio_output_path="temp_audio.wav"):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_output_path, codec='pcm_s16le')
    return audio_output_path
def transcribe_audio(audio_path, model_size="medium", device="auto"):
    model = WhisperModel(model_size, device=device, compute_type="float16")
    segments, _ = model.transcribe(audio_path, beam_size=5)

    sentence_segments = []
    for seg in segments:
        sentence_segments.append({
            "start_time": float(seg.start),
            "end_time": float(seg.end),
            "text": seg.text.strip()
        })
    return sentence_segments

def create_metadata(video_id, transcript_segments):
    metadata = []
    for idx, segment in enumerate(transcript_segments):
        entry = {
            "video_id": video_id,
            "segment_index": idx,
            "start_time": segment["start_time"],
            "end_time": segment["end_time"],
            "text": segment["text"]
        }
        metadata.append(entry)
    return metadata

def generate_caption_metadata(video_path, model_size="medium", device="auto"):
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    print(f"\n📥 Processing video: {video_id}")

    audio_path = extract_audio(video_path)
    print("🔊 Audio extracted.")

    transcript_segments = transcribe_audio(audio_path, model_size, device)
    print(f"📝 Transcribed {len(transcript_segments)} sentence segments.")

    metadata = create_metadata(video_id, transcript_segments)
    return metadata

video_path = "/content/Machine Learning in 3 Minutes.mp4"
caption_metadata = generate_caption_metadata(video_path, model_size="medium", device="cuda")

import pandas as pd
df = pd.DataFrame(caption_metadata)
df.head(10)



📥 Processing video: Machine Learning in 3 Minutes
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
🔊 Audio extracted.
📝 Transcribed 35 sentence segments.


,video_id,segment_index,start_time,end_time,text
0,Machine Learning in 3 Minutes,0,0.00,6.42,Hi! Welcome to E2Ecloud's machine learning ser...
1,Machine Learning in 3 Minutes,1,6.42,12.52,in 3 minutes. Any AI algorithm is really just ...
2,Machine Learning in 3 Minutes,2,12.52,18.36,"inputs, perform hundreds of equations on those..."
3,Machine Learning in 3 Minutes,3,18.36,24.22,Each one of these circles represents a math eq...
4,Machine Learning in 3 Minutes,4,24.22,29.82,"a variable, or weight, that is, learned, over ..."
5,Machine Learning in 3 Minutes,5,29.82,35.06,allow for different strengths of relations bet...
6,Machine Learning in 3 Minutes,6,35.06,40.14,that can help a neural network figure out a fi...
7,Machine Learning in 3 Minutes,7,40.14,45.90,"stronger the connection, the higher the weight..."
8,Machine Learning in 3 Minutes,8,45.90,51.26,weights that give a machine learning algorithm...
9,Machine Learning in 3 Minutes,9,51.26,57.42,network needs data. The machine learning algor...


In [ ]:
import os
import json
import logging
from tqdm import tqdm
from faster_whisper import WhisperModel

def transcribe_and_store(video_name, audio_file_path, json_output_path):
    model = WhisperModel("large-v3", compute_type="float16")
    model.logger.setLevel(logging.WARNING)

    print(f"[INFO] Transcribing audio from {video_name} ...")

    segments, info = model.transcribe(audio_file_path)

    transcript_data = []
    for segment in segments:
        transcript_data.append({
            "text": segment.text.strip(),
            "start_time": round(segment.start, 2),
            "end_time": round(segment.end, 2)
        })

    os.makedirs(os.path.dirname(json_output_path), exist_ok=True)
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump({
            "video_name": video_name,
            "segments": transcript_data
        }, f, indent=4, ensure_ascii=False)

    print(f"[INFO] Transcription complete. Data stored at: {json_output_path}")

video_path = "/content/Machine Learning in 3 Minutes.mp4"
audio_path = "/content/temp_audio.wav"
json_output_path = "/content/transcripts/ml_video_transcript.json"

transcribe_and_store(video_name, audio_path, json_output_path)


[INFO] Transcribing audio from ml_video ...
[INFO] Transcription complete. Data stored at: /content/transcripts/ml_video_transcript.json


In [ ]:
def extract_audio(video_path, audio_output_path="temp_audio.wav"):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_output_path, codec='pcm_s16le')
    return audio_output_path

#  Transcribe using Whisper
def transcribe_audio(audio_path, model_size="medium", device="auto"):
    model = WhisperModel(model_size, device=device, compute_type="float16")
    segments, _ = model.transcribe(audio_path, beam_size=5)

    raw_segments = []
    for seg in segments:
        raw_segments.append({
            "start_time": float(seg.start),
            "end_time": float(seg.end),
            "text": seg.text.strip()
        })
    return raw_segments

(15-20 sec chunks)
def merge_segments(segments, max_duration=20.0):
    merged = []
    temp_text = ""
    temp_start = segments[0]["start_time"]
    temp_end = segments[0]["end_time"]

    for seg in segments:
        duration = temp_end - temp_start
        if duration < max_duration:
            temp_text += " " + seg["text"]
            temp_end = seg["end_time"]
        else:
            merged.append({
                "start_time": temp_start,
                "end_time": temp_end,
                "text": temp_text.strip()
            })
            temp_text = seg["text"]
            temp_start = seg["start_time"]
            temp_end = seg["end_time"]

    # Append last chunk
    if temp_text:
        merged.append({
            "start_time": temp_start,
            "end_time": temp_end,
            "text": temp_text.strip()
        })

    return merged

def generate_caption_metadata(video_path, model_size="medium", device="auto", max_duration=20.0):
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    print(f"\n📥 Processing video: {video_id}")

    audio_path = extract_audio(video_path)
    print("🔊 Audio extracted.")

    raw_segments = transcribe_audio(audio_path, model_size, device)
    print(f"📝 Transcribed {len(raw_segments)} raw segments.")

    merged_segments = merge_segments(raw_segments, max_duration=max_duration)
    print(f"📚 Merged into {len(merged_segments)} context-rich segments (~{max_duration}s each).")

    metadata = []
    for seg in merged_segments:
        metadata.append({
            "video_id": video_id,
            "start_time": round(seg["start_time"], 2),
            "end_time": round(seg["end_time"], 2),
            "text": seg["text"]
        })

    return metadata

# Run the Pipeline
video_path = "/content/Machine Learning in 3 Minutes.mp4"
caption_metadata = generate_caption_metadata(video_path, model_size="medium", device="cuda", max_duration=20.0)

output_json_path = "/content/ml_video_transcript_metadata.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(caption_metadata, f, indent=4, ensure_ascii=False)

print(f"✅ Metadata saved to: {output_json_path}")

import pandas as pd
df = pd.DataFrame(caption_metadata)
df.head(10)



📥 Processing video: Machine Learning in 3 Minutes
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
🔊 Audio extracted.
📝 Transcribed 35 raw segments.
📚 Merged into 9 context-rich segments (~20.0s each).
✅ Metadata saved to: /content/ml_video_transcript_metadata.json


,video_id,start_time,end_time,text
0,Machine Learning in 3 Minutes,0.00,24.22,Hi! Welcome to E2Ecloud's machine learning ser...
1,Machine Learning in 3 Minutes,24.22,45.90,"a variable, or weight, that is, learned, over ..."
2,Machine Learning in 3 Minutes,45.90,67.70,weights that give a machine learning algorithm...
3,Machine Learning in 3 Minutes,67.70,88.98,to test how well their algorithm is able to pe...
4,Machine Learning in 3 Minutes,88.98,111.90,This process continues layer after layer until...
5,Machine Learning in 3 Minutes,111.90,133.18,The real magic of a neural network happens aft...
6,Machine Learning in 3 Minutes,133.18,153.98,which is an indicator of how well or poorly th...
7,Machine Learning in 3 Minutes,153.98,176.42,"the model has been adjusted, the training proc..."
8,Machine Learning in 3 Minutes,176.42,194.94,value and optimizer to go back and fix the wei...
